In [1]:
from __future__ import print_function
import numpy as np
from time import time
from multiprocessing import Process, Queue

In [2]:
def simple_job(out_q, data, device, name):
    # Intra-process Imports.
    import tensorflow as tf
    from tensorflow.python.ops import variable_scope as vs
    
    # Intra-process Computation.
    graph = tf.Graph()
    with graph.as_default():
        with tf.device(device):
            batch = tf.placeholder(tf.float32, [2, 4], name='batch_data_matrix')
            with vs.variable_scope('full_con_1') as scope:
                W_a = tf.get_variable('W_a', initializer=tf.random_normal([4, 8], stddev=0.1))
                b_a = tf.get_variable('b_a', initializer=tf.random_normal([8], stddev=0.1))
                activation = tf.sigmoid(tf.matmul(batch, W_a) + b_a)
    
    # Intra-process Session Manager.
    with tf.Session(graph=graph) as sess:
        sess.run(tf.initialize_all_variables())
        affine_out = sess.run(activation, {batch: data})
    
    # Intra-process Result Collection.
    out_q.put({name: affine_out})

In [3]:
def run_twin(data):
    # Gather data and Book-keeping.
    out_q = Queue()
    
    # Build the processes.
    task_one = Process(target=simple_job, args=(out_q, data, '/cpu:0', 'task_one'))
    task_two = Process(target=simple_job, args=(out_q, data, '/gpu:0', 'task_two'))
    
    # Initialize processes.
    task_one.start()
    task_two.start()
    
    # Collect output.
    results = {}
    results.update(out_q.get())
    results.update(out_q.get())
        
    # Terminate processes.
    task_one.join()
    task_two.join()
    
    return data, results

In [4]:
data = np.random.randn(2, 4).astype(np.float32)

In [ ]:
t = time()
res = run_twin(data)
print("Twin concurrent jobs took %.6fs" %(time() - t))

Process Process-2:
Traceback (most recent call last):
  File "/Users/PragaashP/anaconda2/envs/tensorflow/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/PragaashP/anaconda2/envs/tensorflow/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-f7d5b37e35bb>", line 19, in simple_job
    affine_out = sess.run(activation, {batch: data})
  File "/Users/PragaashP/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 717, in run
    run_metadata_ptr)
  File "/Users/PragaashP/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 915, in _run
    feed_dict_string, options, run_metadata)
  File "/Users/PragaashP/anaconda2/envs/tensorflow/lib/python2.7/site-packages/tensorflow/python/client/session.py", line 965, in _do_run
    target_list, options, run_metadata)
  File "/Users/PragaashP/

In [33]:
map(lambda x: x.shape, res.values())

[(2, 8), (2, 8)]

In [8]:
data

array([[ 0.24282973, -0.05691367,  0.86136663,  0.81268179],
       [ 0.61341918,  0.28797001, -0.95122713, -0.08148833]], dtype=float32)

In [4]:
class SimpleQueue(object):
    def __init__(self):
        self.data = []
        
    def put(self, item):
        self.data.append(item)

In [9]:
A = SimpleQueue()
t = time()
simple_job(A, data, '/gpu:0', 'task_one')
print("Single Job took %.6fs" %(time() - t))

Single Job took 0.492966s
